In [10]:
import itertools
import math
import string
from copy import deepcopy

import blosum as bl
import numpy as np
from enum import IntEnum

In [11]:
class Score(IntEnum):
    MATCH = 1
    MISMATCH = 0
    GAP = -1

# Assigning the constant values for the traceback
class Trace(IntEnum):
    STOP = 0
    LEFT = 1
    UP = 2
    DIAGONAL = 3

# Implementing the Smith Waterman local alignment
def smith_waterman(seq1, seq2):
    # Generating the empty matrices for storing scores and tracing
    row = len(seq1) + 1
    col = len(seq2) + 1
    matrix = np.zeros(shape=(row, col))
    tracing_matrix = np.zeros(shape=(row, col))

    # Initialising the variables to find the highest scoring cell
    max_score = -1
    max_index = (-1, -1)

    # Calculating the scores for all cells in the matrix
    for i in range(1, row):
        for j in range(1, col):
            # Calculating the diagonal score (match score)
            match_value = Score.MATCH if seq1[i - 1] == seq2[j - 1] else Score.MISMATCH
            diagonal_score = matrix[i - 1, j - 1] + match_value

            # Calculating the vertical gap score
            vertical_score = matrix[i - 1, j] + Score.GAP

            # Calculating the horizontal gap score
            horizontal_score = matrix[i, j - 1] + Score.GAP

            # Taking the highest score
            matrix[i, j] = max(0, diagonal_score, vertical_score, horizontal_score)

            # Tracking where the cell's value is coming from
            if matrix[i, j] == 0:
                tracing_matrix[i, j] = Trace.STOP

            elif matrix[i, j] == horizontal_score:
                tracing_matrix[i, j] = Trace.LEFT

            elif matrix[i, j] == vertical_score:
                tracing_matrix[i, j] = Trace.UP

            elif matrix[i, j] == diagonal_score:
                tracing_matrix[i, j] = Trace.DIAGONAL

            # Tracking the cell with the maximum score
            if matrix[i, j] >= max_score:
                max_index = (i,j)
                max_score = matrix[i, j]

    # Initialising the variables for tracing
    aligned_seq1 = ""
    aligned_seq2 = ""
    current_aligned_seq1 = ""
    current_aligned_seq2 = ""
    (max_i, max_j) = max_index

    # Tracing and computing the pathway with the local alignment
    while tracing_matrix[max_i, max_j] != Trace.STOP:
        if tracing_matrix[max_i, max_j] == Trace.DIAGONAL:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = seq2[max_j - 1]
            max_i = max_i - 1
            max_j = max_j - 1

        elif tracing_matrix[max_i, max_j] == Trace.UP:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = '-'
            max_i = max_i - 1

        elif tracing_matrix[max_i, max_j] == Trace.LEFT:
            current_aligned_seq1 = '-'
            current_aligned_seq2 = seq2[max_j - 1]
            max_j = max_j - 1

        aligned_seq1 = aligned_seq1 + current_aligned_seq1
        aligned_seq2 = aligned_seq2 + current_aligned_seq2

    # Reversing the order of the sequences
    aligned_seq1 = aligned_seq1[::-1]
    aligned_seq2 = aligned_seq2[::-1]

    return aligned_seq1, aligned_seq2, max_score

Вход: слово длины 4 по дефолту но с изменением, нуклеотидные последовательности,
ВЫход: упорядоченный список диагоналей с наибольшим числом совпадений
сравнение индексов (длина слова, последовательности)

In [13]:
def to_map(seq: string, k: int):
    dict = {}
    for i in range(len(seq) - k + 1):
        key = seq[i : i + k]
        if dict.get(key) is None:
            dict[key] = []
        dict[key].append(i)
    return dict

In [14]:
def sub_maps_diff_pos(map_1, map_2):
    dist = {}
    for k1 in map_1.keys(): #k1 - 'AA'; val1 - [1, 2, 3]
        if map_2.get(k1) is None:
            continue

        for pos1 in map_1[k1]:
            for pos2 in map_2[k1]:
                delta = pos2 - pos1
                if dist.get(delta) is None:
                    dist[delta] = []
                dist[delta].append(pos1)


    res = []
    for gap in dist:
        li = dist[gap]
        res.append(Diag(convert_diag_to_xy(gap, li)))

    return res

def cutt_off(dist, count=1):
    res = {}
    for key in dist.keys():
        if len(dist[key]) > count:
            res[key] = dist[key]
    return res


In [15]:
def sub_maps(map_1, map_2):
    # dict[subsequence] = set(distance(subsequence))
    dist = {}

    for k1 in map_1.keys():
        if map_2.get(k1) is None:
            continue
        if dist.get(k1) is None:
            dist[k1] = []
        for pos1 in map_1[k1]:
            for pos2 in map_2[k1]:
                dist[k1].append(pos2 - pos1)
    return dist

In [16]:
def convert_to_matrix(diffs, n1, n2):
    res = np.zeros((n1, n2))
    for gap in diffs:
        for i, pos1 in enumerate(diffs[gap]):
            pos2 = pos1 + gap
            res[pos1][pos2] = 1
    return res


In [130]:
class Diag:
    def is_straight(self):
        return (self.y_max - self.y_min) == (self.x_max - self.x_min)

    def __init__(self, li):
        self.x_min = li[0]
        self.y_min = li[1]
        self.x_max = li[2]
        self.y_max = li[3]

    def __add__(self, diag):
        return Diag([min(self.x_min, diag.x_min), min(self.y_min, diag.y_min), max(diag.x_max, self.x_max), max(diag.y_max, self.y_max)])

    def __len__(self):
        return self.x_max - self.x_min + self.y_max - self.y_min

    def to_positions(self):
        return self.x_min, self.y_min, self.x_max, self.y_max

    def __getitem__(self, item):
        x = self.x_min + item
        y = self.y_min + item
        if x > self.x_max or y > self.y_max:
            raise KeyError
        return Diag([x, y, x, y])

    def __eq__(self, other):
        return self.x_min == other.x_min and self.y_min == other.y_min and self.x_max == other.x_max and self.y_max == other.y_max

    def __le__(self, other):
        return len(self) <= len(other)

    def __lt__(self, other):
        return len(self) < len(other)

    def __hash__(self):
        return hash(repr(self))

    def __str__(self):
        return f"({self.x_min}, {self.y_min}) -> ({self.x_max}, {self.y_max}): " + ("straight" if self.is_straight() else "not straight")

In [68]:
def convert_diag_to_xy(gap, poss1):
    MAX = float('inf')
    MIN = float('-inf')
    x_min = MAX
    x_max = MIN
    y_min = MAX
    y_max = MIN
    for i, pos1 in enumerate(poss1):
        pos2 = pos1 + gap
        x_min = min(x_min, pos1)
        x_max = max(x_max, pos1)
        y_min = min(y_min, pos2)
        y_max = max(y_max, pos2)
    return x_min, y_min, x_max, y_max

def get_top_10(dist):
    res = sorted(dist)
    return res[:10:-1]

In [119]:
def find_best_for_diag(diag_scores_old, gap):
    concatted = False
    diag_scores_new = {}
    diag_scores = deepcopy(diag_scores_old)
    for d1 in diag_scores:
        MAX = float('-inf')
        best_d2 = None
        for d2 in diag_scores:
            if not d1.__eq__(d2) and d2.x_min >= d1.x_min and d2.y_min >= d1.y_min:
                # чтобы вторая диагоналей была не выше и не левее первой
                if d2.x_min  <= d1.x_max:
                    dist = d2.y_min - d1.y_min
                elif d2.y_min <= d1.y_max:
                    dist = d2.x_min - d1.x_min
                else:
                    dist = d2.y_min - d1.y_max + d2.x_min - d1.x_max
                res = diag_scores[d1] + diag_scores[d2] + dist * gap
                if res > diag_scores[d1] and res > diag_scores[d2] and res > MAX:
                    MAX = res
                    best_d2 = d2

        if best_d2 is not None:
            diag_scores_new[best_d2] = float('-inf')
            diag_scores_new[d1] = float('-inf')
            concatted = True
            diag_scores_new[(d1 + best_d2)] = MAX
        else:
            diag_scores_new[d1] = diag_scores[d1]
    return concatted, diag_scores_new

In [143]:
def read_fasta(path):
    res = {}
    s1_name, s1 = '', ''
    f = open(path, 'r')
    arr = f.read().split('>')
    for s in arr:
        ss = s.split('\n')
        sss = ''
        for i in range(1, len(ss)):
            sss += ss[i]
        if ss[0] != '':
            if len(s1_name) == 0:
                s1_name = ss[0]
                s1 = sss
            else:
                res[ss[0]] = sss
    return s1_name, s1, res

In [146]:
def get_best_diagonals_concatenation_with_scores(str1, str2, substr_len, gap, blosum_cf):
    map_1 = to_map(str1, substr_len)
    map_2 = to_map(str2, substr_len)
    diags = get_top_10(sub_maps_diff_pos(map_1, map_2))
    diffs2 = []
    blosum_matrix = bl.BLOSUM(62)
    diag_scores = {}

    for d1 in diags:
        diag_scores[d1] = 0.0

        for i in range(d1.x_max - d1.x_min):
            len_1_diag = d1[i]
            diag_scores[d1] += blosum_matrix[str1[len_1_diag.x_min] + str2[len_1_diag.y_min]]
        if diag_scores[d1] < blosum_cf:
            del diag_scores[d1]
        else:
            diffs2.append(d1)


    concatted = True
    while concatted:
        concatted, diag_scores = find_best_for_diag(diag_scores, gap)

    return diag_scores


def find_best_score(diagonals_scores):
    MAX_SCORE = float('-inf')
    MAX_KEY = None
    for keys in diagonals_scores:
        if diagonals_scores[keys] > MAX_SCORE:
            MAX_SCORE = diagonals_scores[keys]
            MAX_KEY = keys
    return MAX_KEY, MAX_SCORE

def operate(seq1, seq2, substr_len, gap, blosum_cf):
    res = get_best_diagonals_concatenation_with_scores(seq1, seq2, substr_len, gap, blosum_cf)
    best_diag, best_score = find_best_score(res)
    s1_1, s2_1, s1_2, s2_2 = best_diag.to_positions()
    return smith_waterman(seq1[s1_1 : s1_2 + 1], seq2[s2_1 : s2_2 + 1])


def operate_fasta(path, substr_len=2, gap=-10, blosum_cf=5):
    s1_name, s1, m = read_fasta(path)
    print(s1_name)
    for key in m:
        if key != s1_name:
            s2_name, s2 = key, m[key]
            al_1, al_2, score = operate(s1, s2, substr_len, gap, blosum_cf)
            print(f"\tto { s2_name}\n\n\t{al_1}\n\t{al_2}\n\t\tScore:{score}\n{'-' * 15}")

P35858.1 RecName: Full=Insulin-like growth factor-binding protein complex acid labile subunit; Short=ALS; Flags: Precursor
	to tr|Q71RY9|Q71RY9_AZOVI Green fluorescence protein OS=Azotobacter vinelandii OX=354 GN=289Gfp PE=4 SV=1

	FE
	FE
		Score:2.0
---------------
	to sp|P42212|GFP_AEQVI Green fluorescent protein OS=Aequorea victoria OX=6100 GN=GFP PE=1 SV=1

	FE
	FE
		Score:2.0
---------------
	to tr|A0A2T4CL13|A0A2T4CL13_9GAMM Uncharacterized protein OS=Alcanivorax venustensis OX=172371 GN=C9924_00300 PE=4 SV=1

	FE
	FE
		Score:2.0
---------------
	to tr|Q8GHE2|Q8GHE2_AZOVI Green fluorescence protein OS=Azotobacter vinelandii OX=354 GN=2289Gfp PE=1 SV=1

	FE
	FE
		Score:2.0
---------------
	to tr|W6KDG8|W6KDG8_NICBE Green fluorescent protein OS=Nicotiana benthamiana OX=4100 GN=gfp PE=4 SV=1

	FE
	FE
		Score:2.0
---------------
	to tr|A0A2S8V8C2|A0A2S8V8C2_9BACI Uncharacterized protein OS=Bacillus sp. MYb78 OX=1827288 GN=CQ064_32485 PE=4 SV=1

	FE
	FE
		Score:2.0
---------------
	to

In [27]:
operate_fasta('seqdump.txt', substr_len=2, gap=-10, blosum_cf=5)